In [ ]:
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pylab as plt
import seaborn as sbn
import pickle

from scipy.stats import ks_2samp
from sklearn.preprocessing import MinMaxScaler, StandardScaler,PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
from sklearn.linear_model import LinearRegression,LogisticRegression
from itertools import permutations, combinations

In [ ]:
path_train1 = 'data_base_demand_train_1_L_0.01.csv' #upload data with noleak case
path_train2 = 'data_base_demand_train_2_L_0.01.csv'  #upload data with noleak case
path_train3 = 'data_base_demand_train_3_L_0.01.csv'  #upload data with noleak case
path_test_base_demand = 'data_base_demand_test_L_0.01.csv' #upload data with noleak case

path_leak_train  = 'leak_train_L_0.01.csv' #upload data with leak case
path_leak_test = 'leak_test_L_0.01.csv'    #upload data with leak case

In [ ]:
# Proxy for historical observations
df_train_reg_model = pd.read_csv(path_train1)
df_train_class_model_ref = pd.read_csv(path_train2)
df_train_class_model_rec = pd.read_csv(path_train3)

# Proxy for recent observations, those need to be tested for leak
df_test_class_model = pd.read_csv(path_test_base_demand)

leak_training = pd.read_csv(path_leak_train)
leak_testing = pd.read_csv(path_leak_test)

In [ ]:
leak_training.head()

In [ ]:
def training_data(df,link_names,head_names):
    
    data_flow = np.array(df[link_names])  # convering to litres per sec 
    data_head = np.array(df[head_names])
    

    train_out= data_head[:,0] - data_head[:,1] # deltaH
    train_in = data_flow                       # flow1, flow2
    
    return train_in, train_out

In [ ]:
# Manually define a sensor list

sensor_list = [[1,1],[2,2],[8,8]]

sen_nums = np.arange(1,len(sensor_list)+1)
combs = list(combinations(sen_nums,2))
print(sen_nums)
combs

In [ ]:
#chk how combs can help in iterating through all possible combinations of sensors
i=2
print(len(combs))
print(combs[i])
print('first sensor is ',combs[i][0])
print('second sensor is ',combs[i][1])

In [ ]:
# function to extract input (flow1, flow2) and output (deltaH) data from datasets prepared earlier
# data without leaks

def data_in_out_noleak(sensor_pair,df):
    
    h1= sensor_list[sensor_pair[0]-1][0]
    h2= sensor_list[sensor_pair[1]-1][0]
    f1= sensor_list[sensor_pair[0]-1][1]
    f2= sensor_list[sensor_pair[1]-1][1]
    
    link_name = ['Link_flow'+str(f1),'Link_flow'+str(f2)] 
    node_name = ['Node_head'+str(h1),'Node_head'+str(h2)]
    
    data_in, data_out = training_data(df,link_name,node_name)
    
    return data_in, data_out 

In [ ]:
# function to extract input and output data with leak

def data_in_out_withleak(sensor_pair,df):
    
    h1= sensor_list[sensor_pair[0]-1][0]
    h2= sensor_list[sensor_pair[1]-1][0]
    f1= sensor_list[sensor_pair[0]-1][1]
    f2= sensor_list[sensor_pair[1]-1][1]
    
    link_name_leak = ['Link_flow'+str(f1),'Link_flow'+str(f2)]
    node_name_leak = ['Node_head'+str(h1),'Node_head'+str(h2)]    
    data_in, data_out = training_data(df,link_name_leak,node_name_leak)
    
    return data_in, data_out

In [ ]:
for comb in combs:    
    xtrain, ytrain = data_in_out_noleak(comb,df_train_reg_model)    
    poly = PolynomialFeatures(degree=2)
    x_train_poly = poly.fit_transform(xtrain)    
    lin_model = LinearRegression()
    lin_model.fit(x_train_poly,ytrain)
    
    pkl_filename = 'linmodel'+str(comb[0])+str(comb[1])+'.pkl'
    with open(pkl_filename, 'wb') as file:
        pickle.dump(lin_model, file)

In [ ]:
# generate column names
colnames = []
for comb in combs:
    n1='obs'+str(comb[0])+str(comb[1])
    n2='prd'+str(comb[0])+str(comb[1])
    n3='stat'+str(comb[0])+str(comb[1])
    n4='pval'+str(comb[0])+str(comb[1])
    colnames.extend([n1,n2,n3,n4])
    

In [ ]:
def datafortrees(numcases,df_recent,sample_len,casetype):
    
    df_cases = pd.DataFrame(columns=colnames)
    #fracsize_recent=sample_len/len(df_recent)
    fracsize_recent=0.3
    print(fracsize_recent)
    
    fracsize_reference = sample_len/len(df_train_class_model_ref)
    
    # first loop to randomly select a sample test set

    for i in range(numcases):
        df_recent_sample = df_recent.sample(frac=fracsize_recent)
        df_reference_sample = df_train_class_model_ref.sample(frac=fracsize_reference)
        
        # second loop to cover all possible leak combinations

        comb_data = []
        for comb in combs:
            if casetype=='noleak':
                xtest_rec, ytest_rec = data_in_out_noleak(comb,df_recent_sample)
            else:
                xtest_rec, ytest_rec = data_in_out_withleak(comb,df_recent_sample)
                
            xtest_ref, ytest_ref = data_in_out_noleak(comb,df_reference_sample)
            

            # load the linear regression model, make predictions and store results
            pkl_filename = 'linmodel'+str(comb[0])+str(comb[1])+'.pkl'

            with open(pkl_filename, 'rb') as file:
                lin_model = pickle.load(file)

            xtest_ref_poly = poly.fit_transform(xtest_ref)
            xtest_rec_poly = poly.fit_transform(xtest_rec) 
            pred_ref = lin_model.predict(xtest_ref_poly).reshape(-1)
            pred_rec = lin_model.predict(xtest_rec_poly).reshape(-1)
            error_ref = (ytest_ref-pred_ref)
            error_rec = (ytest_rec-pred_rec)
            stat,pval = ks_2samp(error_ref,error_rec) #to test the model runs fine
            comb_list = [np.mean(error_ref),np.mean(error_rec),stat,pval]            
            comb_data.extend(comb_list)
            
        comb_series = pd.Series(comb_data,index=df_cases.columns)
        #print(type(comb_series))
        comb_series_1 = pd.DataFrame(comb_series)
        comb_series_2 = comb_series_1.T
        
        
        df_cases=pd.concat([df_cases, comb_series_2])
        #print(type(df_cases))
       
        
      
    return df_cases

In [ ]:
leak_training.leak_link.unique()


In [ ]:
## Defining size of training set. Prefer choosing a multiple of 6 
import warnings
warnings.filterwarnings("ignore")
num_samples = 100
num_training_samples_each = 20000
# num_training_sample_total = 3000
# num_training_sample_leak = int(num_training_sample_total/6)
# num_training_sample_noleak = num_training_sample_total - 3* (num_training_sample_leak)
#warning('off')
class_train_noleak = datafortrees(num_training_samples_each,df_train_class_model_rec,num_samples,'noleak')

class_train_noleak['leak']=0
class_train_noleak['leak_num']=0

class_train_all_leaks = pd.DataFrame(columns = class_train_noleak.columns)
# loop through all the links with leakage and prepare the dataframe
for i in leak_training.leak_link.unique():
    print(i)
    leak_df = leak_training[leak_training.leak_link == i].reset_index(drop=True)
    print(len(leak_df))
    class_train = datafortrees(num_training_samples_each,leak_df,num_samples,'leak')
    class_train['leak']=1
    class_train['leak_num']=i
    #class_train_all_leaks = class_train_all_leaks.append(class_train)
    class_test_all_leaks=pd.concat([class_train_all_leaks,class_train], ignore_index=True)

data_train_classification = pd.concat([class_train_noleak,class_train_all_leaks],axis=0)

In [ ]:
num_samples= 100
num_test_samples_each = 500
# num_test_sample_leak = int(num_test_sample_total/6)
# num_test_sample_noleak = num_test_sample_total - 3* (num_test_sample_leak)

class_test_noleak = datafortrees(num_test_samples_each,df_test_class_model,num_samples,'noleak')
class_test_noleak['leak']=0
class_test_noleak['leak_num']=0

class_test_all_leaks = pd.DataFrame(columns = class_test_noleak.columns)
# loop through all the links with leakage and prepare the dataframe
for i in leak_testing.leak_link.unique():
    leak_df = leak_testing[leak_testing.leak_link == i].reset_index(drop=True)
    class_test = datafortrees(num_test_samples_each,leak_df,num_samples,'leak')
    #print((class_test))
    class_test['leak']=1
    class_test['leak_num']=i
    #class_test_all_leaks = class_test_all_leaks.join(class_test)
    #class_test_all_leaks=pd.concat([class_test_all_leaks,class_test],ignore_index=True)
    class_test_all_leaks=pd.concat([class_test_all_leaks,class_test], ignore_index=True)
    #class_test_all_leaks=pd.merge(class_test_all_leaks,class_test, left_index=True, right_index=True)
    
    

data_test_classification = pd.concat([class_test_noleak,class_test_all_leaks],axis=0)

In [ ]:
#cmat_pd.to_csv('pp1.csv')
data_test_classification.to_csv('data_test_classification_cor.csv')

In [ ]:
data_train_classification.to_csv('data_train_classification_cor.csv')